In [2]:
#!/usr/bin/env python3
"""
Created on July 31 2019 18:02:10

@author: Shibabrat Naik
"""

import numpy as np

from matplotlib import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from pylab import rcParams
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['mathtext.rm'] = 'serif'
# mpl.rcParams['font.family'] = 'serif'
# mpl.rcParams['font.serif'] = ['Helvetica']

# plt.style.use('seaborn-white') # use sans-serif fonts

rcParams['figure.figsize'] = 5, 5

label_size = 25 #10, 20
mpl.rcParams['xtick.labelsize'] = label_size
mpl.rcParams['ytick.labelsize'] = label_size
mpl.rcParams['axes.labelsize'] = 25 #, 15

mpl.rcParams['axes.spines.left'] = True   ## display axis spines
mpl.rcParams['axes.spines.bottom'] = True
mpl.rcParams['axes.spines.top'] = True
mpl.rcParams['axes.spines.right'] = True
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 1.0
mpl.rcParams['ytick.major.width'] = 1.0



In [28]:


def pe_DBbath(x, y, x1, y1, params_pe):
    """
    Potential energy function for the 2 DOF model
    x, y: N x N array of position values as meshgrid data
    params: 1 x 4 array of parameters
    """
    if np.size(x) == 1:
        nX = 1
        nY = 1
    else:
        nX = np.size(x,1)
        nY = np.size(x,0)
        x = np.ravel(x, order = 'C')
        y = np.ravel(y, order = 'C')
        x1 = np.ravel(x1, order = 'C')
        y1 = np.ravel(y1, order = 'C')
    
    
    EPSILON_S, D_X, ZETA, LAMBDA = params_pe
    MASS_bath = 1.0;
    ETA1 = 1.0;
    ETA2 = 1.0;
    NB = 2;
    OMEGAC = 1.0;
    OMEGA1 = OMEGAC*np.log((1.0 - 0.5)/NB); 
    CX1 = np.sqrt((2.0*ETA1*OMEGAC)/(np.pi*NB))*OMEGA1;
    CY1 = np.sqrt((2.0*ETA2*OMEGAC)/(np.pi*NB))*OMEGA1; 
    
    
    vy = 4*y**2*(y**2 - 1.0) + EPSILON_S
    vx = D_X*(1.0 - np.exp(-LAMBDA*x))**2
    vyx = 4*y**2*(y**2 - 1)*( np.exp(-ZETA*LAMBDA*x) - 1.0 )
    
    v_bath = 0.5*(OMEGA1*x1 - (CX1*x)/OMEGA1)**2 + 0.5*(OMEGA1*y1 - (CX1*y)/OMEGA1)**2
    
    pe = np.reshape(vyx + vy + vx + v_bath, (nY, nX))
#     print(np.size(vy))
#     print(x,y,vyx)
    
    return pe


def vec_field_DBbath(states, *params):
    
    """
    Hamiltonian vector field for system (De Leon-Berne model) with 
    2 bath modes
    """
    
    MASS_A, MASS_B, EPSILON_S, D_X, ZETA, LAMBDA = params
    MASS_bath = 1.0;
    ETA1 = 1.0;
    ETA2 = 1.0;
    NB = 2;
    OMEGAC = 1.0;
    OMEGA1 = OMEGAC*np.log((1.0 - 0.5)/NB); 
    CX1 = np.sqrt((2.0*ETA1*OMEGAC)/(np.pi*NB))*OMEGA1;
    CY1 = np.sqrt((2.0*ETA2*OMEGAC)/(np.pi*NB))*OMEGA1; 
    
    q1, q2, q3, q4, p1, p2, p3, p4 = states
    
    q1Dot = (p1/MASS_A)
    q2Dot = (p2/MASS_B)
    q3Dot = (p3/MASS_bath) 
    q4Dot = (p4/MASS_bath)
    p1Dot = 2*D_X*LAMBDA*(np.exp(-LAMBDA*q1) - 1)*np.exp(-LAMBDA*q1) + \
            4*ZETA*LAMBDA*q2**2*(q2**2 - 1)*np.exp(-ZETA*LAMBDA*q1) + \
            (CX1/OMEGA1)*( OMEGA1*q3 - (CX1*q1)/OMEGA1 )
    
    p2Dot = -8*(2*q2**3 - q2)*np.exp(-ZETA*LAMBDA*q1) + \
            (CY1/OMEGA1)*( OMEGA1*q4 - (CY1*q2)/OMEGA1 )
    
    p3Dot = OMEGA1*( OMEGA1*q3 - (CX1*q1)/OMEGA1 );
    
    p4Dot = OMEGA1*( OMEGA1*q4 - (CX1*q2)/OMEGA1 );
    
    return np.array([q1Dot, q2Dot, q3Dot, q4Dot, p1Dot, p2Dot, p3Dot, p4Dot])


### Finding the equilibrium points 

In [37]:


from scipy import optimize

"""
ordering of the parameters in the system model: 
mass A, mass B, epsilon, Dx, alpha (z in paper), lambd  
"""
# params =  [1.0, 10, 1.0, 1.5]
# params =  (8.0, 8.0, 1.0, 10, 2.30, 1.95)
# params =  (8.0, 8.0, 1.0, 10, 1.00, 2.00)
# params =  (8.0, 8.0, 1.0, 10, 1.00, 1.50)
# params =  (8.0, 8.0, 1.0, 10, 1.00, 1.30)
# params =  (8.0, 8.0, 1.0, 10, 1.00, 1.00)
# params =  (8.0, 8.0, 1.0, 10, 0.20, 1.00)


eq_pt_1 = optimize.fsolve(vec_field_DBbath, [0, 1, 0, 0, 0, 0, 0, 0], \
                          args = params, xtol = 1e-12, maxfev = 1000)
print(eq_pt_1)

eq_pt_2 = optimize.fsolve(vec_field_DBbath, [0, -1, 0, 0, 0, 0, 0, 0], \
                          args = params, xtol = 1e-12)
print(eq_pt_2)

eq_pt_3 = optimize.fsolve(vec_field_DBbath, [0.1, 0.1, 0, 0, 0, 0, 0, 0], \
                          args = params)
print(eq_pt_3)


totalEnergyEqPt1 = pe_DBbath(eq_pt_1[0], eq_pt_1[1], eq_pt_1[2], eq_pt_1[3], params[2:])
print(totalEnergyEqPt1)

totalEnergyEqPt3 = pe_DBbath(eq_pt_3[0], eq_pt_3[1], eq_pt_3[2], eq_pt_3[3], params[2:])
print(totalEnergyEqPt3)




[-1.12782753e-01  7.07106781e-01  4.58999591e-02 -2.87776025e-01
  8.05149033e-32 -9.56238666e-32  9.12500343e-33 -2.50902704e-33]
[-1.12782753e-01 -7.07106781e-01  4.58999591e-02  2.87776025e-01
 -4.26002885e-38 -9.06128976e-31  4.90073437e-41  9.26618665e-34]
[-1.19336105e-17 -2.93871699e-37  4.85670211e-18  1.19598951e-37
  1.34835553e-49  1.26987482e-50 -9.33144256e-53  4.55018346e-52]
[[-0.37229191]]
[[1.]]
